In [116]:
import pandas as pd
from keras.layers import Input, Embedding, LSTM, Dense, Dot, Softmax, Concatenate, BatchNormalization, Attention
from keras.models import Model
import tensorflow as tf
import numpy as np

In [117]:
basepath = '../../datasets/preprocessed_datasets/gabor/'
product_data = pd.read_pickle(basepath +'product_data.pkl')

In [118]:
product_data = product_data.sample(n=1000)
product_data

,order_id,orderdate,total_order_price,customer__id,product__id,productName,amount,totalPrice,productSizeEu,o_parentId,...,indicatorWedding,indicatorSale,weight__value,color,productType,materialPm,material,line,originCountry,gender
275350,8575223,2022-05-06 09:38:51,120.00,8575226.0,7701784,Sneaker low Rauleder grün,1,120.00,38.5,7697493,...,0.0,1,0.82,11,0,11,3,2,0,0
227862,8690923,2022-05-17 18:26:42,99.95,8690926.0,7595493,Plateau Sandale Glattleder weiß,1,99.95,35.0,7588495,...,0.0,1,0.73,2,0,708,0,2,0,0
458294,9656182,2022-09-26 17:43:27,235.00,1991518.0,8807159,Sneaker low Materialmix Leder blau,1,125.00,38.0,8806628,...,0.0,1,1.26,1,0,224,6,1,1,0
197169,8781175,2022-05-28 09:17:21,179.90,8781179.0,7830981,Plateau Sandale Glattleder braun,1,89.95,39.0,7830528,...,0.0,1,1.01,3,0,0,0,2,0,0
452647,7743246,2022-01-06 18:18:18,120.00,7743244.0,6556896,Plateau Stiefelette Lackleder schwarz,1,120.00,35.5,6551874,...,0.0,1,0.86,0,0,73,4,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7301,8338110,2022-04-11 10:14:27,89.95,8338112.0,556482,Eleganter Pump Glattleder weiß,1,89.95,39.0,337452,...,0.0,0,0.68,2,0,654,0,0,0,0
522912,8146355,2022-03-19 20:21:53,89.95,8089667.0,7789486,Eleganter Ballerina Rauleder Grün,1,89.95,41.0,588301,...,0.0,1,0.68,11,0,11,3,2,1,0
546254,6595200,2021-08-12 19:55:36,158.00,6595203.0,4914593,Sneaker low Materialmix Leder beige,1,79.00,42.5,4913028,...,0.0,1,0.78,5,0,838,6,2,1,0
86212,7556615,2021-11-28 08:57:01,399.75,2123884.0,6114034,Plateau Pumps Textil grau,1,79.95,38.5,3472450,...,0.0,1,0.45,7,0,149,2,1,0,0


In [119]:
product_data

,order_id,orderdate,total_order_price,customer__id,product__id,productName,amount,totalPrice,productSizeEu,o_parentId,...,indicatorWedding,indicatorSale,weight__value,color,productType,materialPm,material,line,originCountry,gender
275350,8575223,2022-05-06 09:38:51,120.00,8575226.0,7701784,Sneaker low Rauleder grün,1,120.00,38.5,7697493,...,0.0,1,0.82,11,0,11,3,2,0,0
227862,8690923,2022-05-17 18:26:42,99.95,8690926.0,7595493,Plateau Sandale Glattleder weiß,1,99.95,35.0,7588495,...,0.0,1,0.73,2,0,708,0,2,0,0
458294,9656182,2022-09-26 17:43:27,235.00,1991518.0,8807159,Sneaker low Materialmix Leder blau,1,125.00,38.0,8806628,...,0.0,1,1.26,1,0,224,6,1,1,0
197169,8781175,2022-05-28 09:17:21,179.90,8781179.0,7830981,Plateau Sandale Glattleder braun,1,89.95,39.0,7830528,...,0.0,1,1.01,3,0,0,0,2,0,0
452647,7743246,2022-01-06 18:18:18,120.00,7743244.0,6556896,Plateau Stiefelette Lackleder schwarz,1,120.00,35.5,6551874,...,0.0,1,0.86,0,0,73,4,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7301,8338110,2022-04-11 10:14:27,89.95,8338112.0,556482,Eleganter Pump Glattleder weiß,1,89.95,39.0,337452,...,0.0,0,0.68,2,0,654,0,0,0,0
522912,8146355,2022-03-19 20:21:53,89.95,8089667.0,7789486,Eleganter Ballerina Rauleder Grün,1,89.95,41.0,588301,...,0.0,1,0.68,11,0,11,3,2,1,0
546254,6595200,2021-08-12 19:55:36,158.00,6595203.0,4914593,Sneaker low Materialmix Leder beige,1,79.00,42.5,4913028,...,0.0,1,0.78,5,0,838,6,2,1,0
86212,7556615,2021-11-28 08:57:01,399.75,2123884.0,6114034,Plateau Pumps Textil grau,1,79.95,38.5,3472450,...,0.0,1,0.45,7,0,149,2,1,0,0


In [120]:
num_dates = len(product_data.orderdate)
num_units = len(product_data.amount)

num_products =  len(product_data.product__id)
display(num_dates, num_products, num_units)

1000

1000

1000

In [122]:
# Input layers
date_inputs = Input(shape=(num_dates,))
product_inputs = Input(shape=(num_products,))

encoding_padding_mask = tf.math.logical_not(tf.math.equal(product_inputs, 0))

# Embedding layers
date_emb = Embedding(num_dates, 16, input_length=num_dates)(date_inputs)
product_emb = Embedding(num_products, 16, input_length=num_products)(product_inputs)

concat_embedding_input = Concatenate(
    name='concat_embedding_input')([date_emb, product_emb])

concat_embedding_input = BatchNormalization(
    name='batchnorm_inputs')(concat_embedding_input)

# LSTM layer
lstm = LSTM(64, return_sequences=True)(concat_embedding_input)

lstm = BatchNormalization(name='batchnorm_lstm')(lstm)

att = Attention(use_scale=False,
                name='attention')(inputs=[lstm, lstm],
                                  mask=[encoding_padding_mask,
                                        encoding_padding_mask])


# Concatenate layer
#concat = Concatenate(axis=2)([context, price_inputs])

# Dense layer
output = Dense(1)(att)
# Define the model
model = Model(inputs=[date_inputs, product_inputs], outputs=output)
# Compile the model
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mse'])

In [ ]:
import datetime

# Convert orderdate column to Unix timestamps
unix_timestamps = [int(datetime.datetime.timestamp(dt)) for dt in product_data['orderdate']]

# Convert Unix timestamps to a numpy array
order_dates = np.array(unix_timestamps)

In [123]:
output_variable = product_data.amount.to_numpy().reshape(-1, 1)
#dataset = tf.data.Dataset.from_tensor_slices()
history = model.fit([order_dates, product_data.product__id], output_variable, epochs=10, batch_size=32)


Epoch 1/10


ValueError: in user code:

    File "/Users/thomashuber/miniconda3/envs/m1-master-thesis/lib/python3.9/site-packages/keras/engine/training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "/Users/thomashuber/miniconda3/envs/m1-master-thesis/lib/python3.9/site-packages/keras/engine/training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/thomashuber/miniconda3/envs/m1-master-thesis/lib/python3.9/site-packages/keras/engine/training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "/Users/thomashuber/miniconda3/envs/m1-master-thesis/lib/python3.9/site-packages/keras/engine/training.py", line 1023, in train_step
        y_pred = self(x, training=True)
    File "/Users/thomashuber/miniconda3/envs/m1-master-thesis/lib/python3.9/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb